In [1]:
import java.nio.file.Paths
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
//now create session
val spark: org.apache.spark.sql.SparkSession = SparkSession
      .builder()
      .appName("Amadeus challenge")
      .config("spark.master", "local")
      .getOrCreate()

import spark.implicits._

Intitializing Scala interpreter ...

Spark Web UI available at http://10.0.0.161:4040
SparkContext available as 'sc' (version = 2.3.1, master = local[*], app id = local-1565520790871)
SparkSession available as 'spark'


2019-08-11 20:53:19 WARN  SparkContext:66 - Using an existing SparkContext; some configuration may not take effect.


import java.nio.file.Paths
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@214d803
import spark.implicits._


In [2]:
val root_path: String = "./";//where to find files
val searches_path: String = root_path + "big_searches.csv";
val df1=spark.read.option("sep", "^").option("header", "true").option("inferSchema", "false").csv(searches_path)

root_path: String = ./
searches_path: String = ./big_searches.csv
df1: org.apache.spark.sql.DataFrame = [Date: string, Time: string ... 43 more fields]


In [3]:
val df1_columns = df1.columns.map(name=>name.trim)
val df_searches = df1.toDF(df1_columns: _*)
df_searches.printSchema

root
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- TxnCode: string (nullable = true)
 |-- OfficeID: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- RoundTrip: string (nullable = true)
 |-- NbSegments: string (nullable = true)
 |-- Seg1Departure: string (nullable = true)
 |-- Seg1Arrival: string (nullable = true)
 |-- Seg1Date: string (nullable = true)
 |-- Seg1Carrier: string (nullable = true)
 |-- Seg1BookingCode: string (nullable = true)
 |-- Seg2Departure: string (nullable = true)
 |-- Seg2Arrival: string (nullable = true)
 |-- Seg2Date: string (nullable = true)
 |-- Seg2Carrier: string (nullable = true)
 |-- Seg2BookingCode: string (nullable = true)
 |-- Seg3Departure: string (nullable = true)
 |-- Seg3Arrival: string (nullable = true)
 |-- Seg3Date: string (nullable = true)
 |-- Seg3Carrier: string (nullable = true)
 |-- Seg3BookingCode: string (null

df1_columns: Array[String] = Array(Date, Time, TxnCode, OfficeID, Country, Origin, Destination, RoundTrip, NbSegments, Seg1Departure, Seg1Arrival, Seg1Date, Seg1Carrier, Seg1BookingCode, Seg2Departure, Seg2Arrival, Seg2Date, Seg2Carrier, Seg2BookingCode, Seg3Departure, Seg3Arrival, Seg3Date, Seg3Carrier, Seg3BookingCode, Seg4Departure, Seg4Arrival, Seg4Date, Seg4Carrier, Seg4BookingCode, Seg5Departure, Seg5Arrival, Seg5Date, Seg5Carrier, Seg5BookingCode, Seg6Departure, Seg6Arrival, Seg6Date, Seg6Carrier, Seg6BookingCode, From, IsPublishedForNeg, IsFromInternet, IsFromVista, TerminalID, InternetOffice)
df_searches: org.apache.spark.sql.DataFrame = [Date: string, Time: string ... 43 more fields]


In [4]:

val searches = sc.textFile(searches_path)//Load RDD 1
val searches_count = searches.count()

searches: org.apache.spark.rdd.RDD[String] = ./big_searches.csv MapPartitionsRDD[10] at textFile at <console>:44
searches_count: Long = 20008000


In [5]:
/* Lets define a function to take each line and return what we want
 * In particular, we want the column 12 with arr_port and 34 with pax
 * VIP: to handle the wrong lines reading (ex. 14 fields) we need to cover that exception, 
 * otherwise it would corrupt the RDD and give many headaches...
 */
def parseLineSearches(line:String) = {
    val fields   = line.split("\\^")
    //check if the line is complete..
    if (fields.length>34){
    val date = fields(0).trim
    val destination = fields(6).trim
    
    if (destination ==" " || date=="") ("KKK","0") else
    (destination, date)
    }
    else ("KKK","0")
}

parseLineSearches: (line: String)(String, String)


In [6]:
val searches01 = searches.map(parseLineSearches).persist()

searches01: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[11] at map at <console>:48


In [7]:
searches01.take(10)

res1: Array[(String, String)] = Array(("Destination","Date"), ("AUH","2013-01-01"), ("MIL","2013-01-01"), ("SFO","2013-01-01"), ("ARN","2013-01-01"), ("MAD","2013-01-01"), ("BLR","2013-01-01"), ("PAR","2013-01-01"), ("DUB","2013-01-01"), ("ACE","2013-01-01"))


In [8]:
val searches02=searches01.filter(e => e._1 == "MAD" || e._1 == "BCN" || e._1 == "AGP").persist()

searches02: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[12] at filter at <console>:50


In [9]:
val searches02=searches01.filter(record => List("MAD","BCN", "AGP").contains(record._1)).persist()//Check this Sets

searches02: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[13] at filter at <console>:50
